# Tarea 4 Modelamiento y Optimización IN3171-1
# Integrantes (sec 1): Adolfo Rojas Valenzuela, Diego E. Cristallini

# Parte 1)

In [ ]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 63.2 MB/s eta 0:00:00


In [ ]:
from gurobipy import Model, GRB

# Crear el modelo
m = Model("Muebles3B")

# Parámetros
H = {'A': (50, 60),
     'B': (42, 48),
     'C': (30, 35)}
h_disponibles = {'A': 40, 'B': 30, 'C': 35}
salario_h = {'A': 3.6, 'B': 4.2, 'C': 5.5}

# Variables de decisión
x = m.addVars(['A', 'B', 'C'], [1, 2], lb=0, name="x")
s = m.addVars([1, 2, 3], vtype=GRB.CONTINUOUS, lb=0, name="s")

# Restricciones
m.addConstr(x.sum('*', 1) == 1, "mesas")
m.addConstr(x.sum('*', 2) == 1, "sillas")

k = 0
for i, (h1, h2) in H.items():
    k += 1
    m.addConstr(h1 * x[i, 1] + h2 * x[i, 2] + s[k] == h_disponibles[i], f"horas_{i}")

# Función objetivo
m.setObjective(
    sum(salario_h[i] * (H[i][0] * x[i, 1] + H[i][1] * x[i, 2]) for i in H),
    GRB.MINIMIZE
)

# Optimizar el modelo
m.optimize()

# Resultados
if m.status == GRB.OPTIMAL:
    print(f"Costo mínimo: ${m.objVal:.2f}")
    for v in m.getVars():
        print(f"{v.varName}: {v.x:.3f}" if v.x != 0 else f"{v.varName}: {v.x:.0f}")
else:
    print("No se encontró una solución óptima")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 9 columns and 15 nonzeros
Model fingerprint: 0x332bff03
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 0 rows and 3 columns
Presolve time: 0.01s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5750000e+02   3.750000e+00   0.000000e+00      0s
       3    3.6725000e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.672500000e+02
Costo mínimo: $367.25
x[A,1]: 0.271
x[A,2]: 0
x[B,1]: 0
x[B,2]: 0.625
x[C,1]: 0.729
x[C,2]: 0.375
s[1]: 26.458
s[2]: 0
s[3]: 0


In [ ]:
from gurobipy import Model, GRB

# Crear el modelo
m = Model("dualMuebles3B")

# Variables de decisión
y1 = m.addVar(name="y1")
y2 = m.addVar(name="y2")
yA = m.addVar(name="yA", lb=0)
yB = m.addVar(name="yB", lb=0)
yC = m.addVar(name="yC", lb=0)

# Restricciones
m.addConstr(y1 - 50*yA <= 180, "constraint1")
m.addConstr(y2 - 60*yA <= 216, "constraint2")
m.addConstr(y1 - 42*yB <= 176.4, "constraint3")
m.addConstr(y2 - 48*yB <= 201.6, "constraint4")
m.addConstr(y1 - 30*yC <= 165, "constraint5")
m.addConstr(y2 - 35*yC <= 192.5, "constraint6")

# Función objetivo
m.setObjective(y1 + y2 - 40*yA - 30*yB - 35*yC, GRB.MAXIMIZE)

# Optimizar el modelo
m.optimize()

# Resultados
if m.status == GRB.OPTIMAL:
    print("Valor óptimo de la función objetivo: ", m.objVal)
    for v in m.getVars():
        print(f"{v.varName}: {v.x}")
else:
    print("No se encontró una solución óptima")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 5 columns and 12 nonzeros
Model fingerprint: 0x435f2962
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve time: 0.02s
Presolved: 6 rows, 5 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+31   5.000000e+30   1.200000e+01      0s
       5    3.6725000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.04 seconds (0.00 work units)
Optimal objective  3.672500000e+02
Valor óptimo de la función objetivo:  367.25
y1: 180.0
y2: 210.0
yA: 0.0
yB: 0.17500000000000013
yC: 0.5


# Parte 2)

In [ ]:
from gurobipy import Model, GRB

# Conjuntos de vértices y aristas
V = [1, 2, 3, 4, 5]
I = [1, 2, 3, 4, 5, 6]
E = [(1, 2), (1, 4), (2, 4), (2, 3), (3, 4), (3, 5)]
Cliques = [[1, 2, 4], [2, 3, 4]]

# Crear el modelo
model = Model('Maximal Independent Set')

# Crear las variables
# x = model.addVars(V, vtype=GRB.BINARY, name="x")
x = model.addVars(V, vtype=GRB.CONTINUOUS, lb=0, name="x")
# s = model.addVars(I, vtype=GRB.CONTINUOUS, lb=0, name="s")

# Establecer la función objetivo
model.setObjective(x.sum(), GRB.MAXIMIZE)

# Agregar las restricciones de adyacencia
k = 0
for (i, j) in E:
    k += 1
    try:
      model.addConstr(x[i] + x[j] + s[k] == 1, f"edge_{i}_{j}")
    except:
      model.addConstr(x[i] + x[j] <= 1, f"edge_{i}_{j}")

# Agregar las restricciones de los cliques
try:
    for C in Cliques:
        model.addConstr(sum(x[i] for i in C) <= 1, f"clique_{'_'.join(map(str, C))}")
except:
    pass

# Optimizar el modelo
model.optimize()

# Imprimir los resultados
if model.status == GRB.OPTIMAL:
    print("Óptimo: $", model.objVal)
    for v in model.getVars():
        if v.x == 0:
            print(f"{v.varName}: 0")
        else:
            print(f"{v.varName}: {v.x}")
else:
    print("No hay óptimo")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 5 columns and 18 nonzeros
Model fingerprint: 0xc5ac3206
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   2.000000e+00   0.000000e+00      0s
       2    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.000000000e+00
Óptimo: $ 2.0
x[1]: 0
x[2]: 0
x[3]: 0
x[4]: 1.0
x[5]: 1.0


In [ ]:
from gurobipy import Model, GRB

# Crear el modelo dual sin clique
dual = Model('Dual')

V = [1, 2, 3, 4, 5, 6]

# Crear las variables
y = dual.addVars(V, vtype=GRB.CONTINUOUS, lb=0, name="y")

# Establecer la función objetivo
dual.setObjective(y.sum(), GRB.MINIMIZE)

# Agregar las restricciones
dual.addConstr(y[1] + y[2] >= 1, 'd1')
dual.addConstr(y[1] + y[3] + y[4] >= 1, 'd2')
dual.addConstr(y[3] + y[5] + y[6] >= 1, 'd3')
dual.addConstr(y[2] + y[4] + y[5] >= 1, 'd4')
dual.addConstr(y[6] >= 1, 'd5')

# Optimizar el modelo
dual.optimize()

# Imprimir los resultados
if dual.status == GRB.OPTIMAL:
    print("Óptimo: $", dual.objVal)
    for v in dual.getVars():
        if v.x == 0:
            print(f"{v.varName}: 0")
        else:
            print(f"{v.varName}: {v.x}")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x5922c23a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 1 columns
Presolve time: 0.03s
Presolved: 3 rows, 5 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   3.000000e+00   0.000000e+00      0s
       3    2.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.04 seconds (0.00 work units)
Optimal objective  2.500000000e+00
Óptimo: $ 2.5
y[1]: 0.5
y[2]: 0.5
y[3]: 0
y[4]: 0.5
y[5]: 0
y[6]: 1.0


# Parte 3)
Mátriz $A$ del primal para el grafo propuesto
$\left(\begin{matrix}
-1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & -1 & 0 & 1 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1 & -1 & 0 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & -1 & -1 & 0 & 1 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & -1 & -1 & 0 & 1 \\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & -1 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{matrix}\right)$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Conjuntos de nodos y aristas
N = ['s', '1', '2', '3', '4', 't']
A = [('s', '1'), ('s', '2'), ('1', '2'), ('1', '3'), ('2', '3'), ('2', '4'), ('3', '4'), ('3', 't'), ('4', 't')]

# Capacidad de las aristas
u = {
    ('1', '3'): 2, ('3', '4'): 2,
    ('2', '3'): 3, ('2', '4'): 3,
    ('s', '1'): 4, ('1', '2'): 4, ('3', 't'): 4,
    ('s', '2'): 5, ('4', 't'): 5
}

# Crear un nuevo modelo
m = gp.Model("max_flow")

# Crear variables de flujo x_ij para cada arco (i, j) en A
x = m.addVars(A, lb=0, name="x")

# Crear la variable de flujo total v
v = m.addVar(name="v")

# Establecer el objetivo: maximizar v
m.setObjective(v, GRB.MAXIMIZE)

# Restricciones de flujo
# Restricción de flujo en el nodo fuente 's'
m.addConstr(v == x[('s', '1')] + x[('s', '2')], "flow_conservation_s")

# Restricciones de conservación de flujo en los nodos intermedios
m.addConstr(x[('s', '1')] == x[('1', '2')] + x[('1', '3')], "flow_conservation_1")
m.addConstr(x[('s', '2')] + x[('1', '2')] == x[('2', '3')] + x[('2', '4')], "flow_conservation_2")
m.addConstr(x[('1', '3')] + x[('2', '3')] == x[('3', '4')] + x[('3', 't')], "flow_conservation_3")
m.addConstr(x[('2', '4')] + x[('3', '4')] == x[('4', 't')], "flow_conservation_4")

# Restricción de flujo en el nodo sumidero 't'
m.addConstr(x[('3', 't')] + x[('4', 't')] == v, "flow_conservation_t")

# Restricciones de capacidad para cada arco
m.addConstrs((x[i, j] <= u[i, j] for i, j in A), "capacity")

# Optimizar el modelo
m.optimize()

# Imprimir los resultados
if m.status == GRB.OPTIMAL:
    print("Optimal value (maximum flow):", v.X)
    for (i, j) in A:
        print(f"Flow on arc ({i}, {j}): {x[i,j].X}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 15 rows, 10 columns and 29 nonzeros
Model fingerprint: 0x7c222b61
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+00]
Presolve removed 12 rows and 4 columns
Presolve time: 0.01s
Presolved: 3 rows, 6 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+00   3.000000e+00   0.000000e+00      0s
       2    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  8.000000000e+00
Optimal value (maximum flow): 8.0
Flow on arc (s, 1): 4.0
Flow on arc (s, 2): 4.0
Flow on arc (1, 2): 2.0
Flow on arc (1, 3): 2.0

In [ ]:
m = gp.Model("dual_max_flow")

# Crear variables duales y_ij para cada restricción de conservación de flujo y z_ij para cada restricción de capacidad
y = m.addVars(N, lb=-GRB.INFINITY, name="y")
z = m.addVars(A, lb=0, name="z")

# Establecer el objetivo: minimizar la suma ponderada de las capacidades u_ij por las variables duales z_ij
m.setObjective(gp.quicksum(u[i, j] * z[i, j] for i, j in A), GRB.MINIMIZE)

# Restricción del valor del flujo total
m.addConstr(y['t'] - y['s'] == 1, "total_flow")

# Restricciones duales para cada arco (i, j)
for i, j in A:
    m.addConstr(y[i] - y[j] + z[i, j] >= 0, f"dual_constraint_{i}_{j}")

# Optimizar el modelo
m.optimize()

# Imprimir los resultados
if m.status == GRB.OPTIMAL:
    print("Optimal value (minimum cost):", m.objVal)
    for i in N:
        print(f"Dual variable y[{i}]: {y[i].X}")
    for (i, j) in A:
        print(f"Dual variable z[{i},{j}]: {z[i,j].X}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 15 columns and 29 nonzeros
Model fingerprint: 0x6f9e50cb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5 rows and 4 columns
Presolve time: 0.01s
Presolved: 5 rows, 11 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.000000e+00   0.000000e+00      0s
       5    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  8.000000000e+00
Optimal value (minimum cost): 8.0
Dual variable y[s]: -1.0
Dual variable y[1]: -1.0
Dual variable y[2]: -1.0
Dual variable y[3]: